In [1]:
from sklearn.neighbors import KNeighborsClassifier
# from sklearn import preprocessing
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [ ]:
import os
from flask import Flask, flash, request, redirect, url_for, render_template

app = Flask(__name__)
upload_path = 'inputs'
app.config['UPLOAD_FOLDER'] = upload_path

In [ ]:
@app.route("/", methods=["GET", "POST"])
def main():
    return render_template('inputForm.html')

@app.route("/get-file", methods=['POST', 'GET'])
def submitted():
    global file_content
    
    if request.method == 'POST':
        result = request.form['file-select']
        
        result = os.path.join(upload_path, result)
        
        text = open(result,'r+')
        content = text.read()
        file_content = result
        text.close()
        
        listPerson = ['Airlangga', 'Hartarto', 'Kevin']
        listOrg = ['ITHB', 'Kemendikbud', 'MK']
        listLocation = ['Jakarta', 'Bandung', 'Istana negara']
        
        print(content)
        return render_template('upload.html', list_person = listPerson, list_org = listOrg, list_loc = listLocation)

<h1>Load hasil baca berita</h1>

In [2]:
def loadPickle(name):
    file = open(name, 'rb')
    data = pickle.load(file)
    file.close()
    
    return data

In [3]:
def listOfDictToDataFrame(listOfDict):
    dataFrame = pd.DataFrame(listOfDict)
    
    x = dataFrame.iloc[:, :-1].values
    y = dataFrame.iloc[:, 4].values
    
    return dataFrame, x, y

<h1>Baca Data Test</h1>

In [4]:
def extFeature(words):
    listOfDictFeature = []
    i = 0
    for i in range(len(words)): #iterasi dari awal tiap kata
        dictFeature = {}
        
        dictFeature = inputToDict(words, i)
        
        listOfDictFeature.append(dictFeature)
        
    return listOfDictFeature

def inputToDict(words, i):
    dictFeature = {}
    
    dictFeature['index'] = i    
    dictFeature['first_upper_case'] = isFirstUpper(words[i])
    dictFeature['prev_word'] = isEdge(i, words)
    dictFeature['word'] = words[i]
    dictFeature['jenis'] = 'o'
    
    return dictFeature

def isFirstUpper(word):
    if word[0].isupper():
        return 'y'
    else:
        return 'n'
    
def isEdge(i, words):
    if i == 0:
        return 'null'
    else:
        return words[i-1]

In [5]:
def readPerWord(file):
    words = []
    f = open(file, 'r') #open file
    for line in f:
        words = line.split(' ') #tiap satu baris split per kata
    
    f.close()
    
    words[len(words)-1] = words[len(words)-1].replace('\n',' ')

    return words

<h1>Proses ==============</h1>

In [6]:
testWord = readPerWord("test.txt")
listOfDictFeature = extFeature(testWord)
panjangTestWord = len(listOfDictFeature)

In [7]:
data = loadPickle("training.pickle")

In [8]:
print(len(data))

3779


In [9]:
print(len(listOfDictFeature))

35


In [10]:
data.extend(listOfDictFeature)

In [11]:
print(len(data), len(data[0]))

3814 5


In [12]:
totalPanjangData = len(data)

In [13]:
dataFrame, x, y = listOfDictToDataFrame(data)

In [14]:
forx = OneHotEncoder(handle_unknown='ignore')
forx.fit(x)
newx = forx.transform(x).toarray()
print(len(newx),len(newx[0]))

3814 3878


<h3>Bangun classifier KNN</h3>

In [15]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(vectorizedX, y, test_size = 0.33, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(newx, y, test_size = panjangTestWord, shuffle=False)

In [16]:
print(len(X_train),len(X_train[0]))
print(len(X_test),len(X_test[0]))
print(len(y_train))
print(len(y_test))

3779 3878
35 3878
3779
35


In [17]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

<h3>Prediksi</h3>

In [18]:
y_pred = classifier.predict(X_test)

In [19]:
print(y_pred)

['o' 'o' 'o' 'o' 'o' 'organization' 'o' 'o' 'o' 'o' 'o' 'o' 'person'
 'person' 'o' 'o' 'o' 'o' 'o' 'o' 'o' 'o' 'organization' 'organization'
 'o' 'o' 'o' 'o' 'o' 'o' 'o' 'o' 'o' 'o' 'o']


In [20]:
# from sklearn.metrics import classification_report, confusion_matrix
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

<h3>Hasil</h3>

In [21]:
wordInvers = forx.inverse_transform(X_test)

In [22]:
person = []
location = []
organization = []
for i in range(len(y_pred)):
    if y_pred[i] == 'person':
        person.append(wordInvers[i][3])
    elif y_pred[i] == 'location':
        location.append(wordInvers[i][3])
    elif y_pred[i] == 'organization':
        organization.append(wordInvers[i][3])

In [23]:
print(person)
print(location)
print(organization)

['Airlangga', 'Hartarto']
[]
['Iskandarsyah', 'Partai', 'Golkar,']


In [ ]:
def prediksi():
    